In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import json
import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import random
import numpy as np


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(43)

In [ ]:
label_mapping = ["flooded",
"non flooded",
"flooded,non flooded",
"Yes",
"No"] +list( map(str, range(51)))

In [ ]:
OLD_FEATURE_PATH = "/content/drive/MyDrive/continual_floodnet_data/floodnet_clip_features"
FEATURE_PATH = "/content/drive/MyDrive/continual_floodnet_data/floodnet_clip_features"

In [ ]:
len(os.listdir(os.path.join(FEATURE_PATH, "Images/Train_Image")))

1448

In [ ]:
all_images_features = dict()
for imagename in os.listdir(os.path.join(FEATURE_PATH, "Images/Train_Image")):
    all_images_features[imagename.replace(".pt", ".JPG")] = torch.load(os.path.join(os.path.join(FEATURE_PATH, "Images/Train_Image"), imagename))

all_questions_features = dict()
for imagename in os.listdir(os.path.join(OLD_FEATURE_PATH, "Questions")):
    all_questions_features[imagename.replace(".pt", "").lstrip("Questions")] = torch.load(os.path.join(os.path.join(OLD_FEATURE_PATH, "Questions"), imagename))


In [ ]:
all_questions_features.keys()

dict_keys(['How many buildings are flooded in this image?', 'How many buildings are flooded?', 'How many buildings are in the image?', 'How many buildings are in this image?', 'How many buildings are non flooded in this image?', 'How many buildings are non flooded?', 'How many buildings can be seen in the image?', 'How many buildings can be seen in this image?', 'How many flooded buildings can be seen in this image?', 'How many non flooded buildings can be seen in this image?', 'Is the entire road flooded?', 'Is the entire road non flooded?', 'What is the condition of road?', 'What is the condition of the road in this image?', 'What is the overall condition of the given image?'])

In [ ]:
class VQADataset(Dataset):
    def __init__(self, qdict, label_mapping=label_mapping):
        self.qdict = qdict
        self.label_mapping = label_mapping
        self.reset_index()

    def reset_index(self):
        new_qdict = dict()
        for idx, value in enumerate(self.qdict.values()):
            new_qdict[idx] = value
        self.qdict = new_qdict

    def __len__(self):
        return len(self.qdict.keys())
    
    def __getitem__(self, idx):
        # "0":{"Image_ID":"10165.JPG","Question":"What is the overall condition of the given image?","Ground_Truth":"flooded","Question_Type":"Condition_Recognition"}
        row = self.qdict[idx]
        img_feat = all_images_features[row["Image_ID"]]
        q_feat = all_questions_features[row["Question"]]
        
        return img_feat, q_feat.squeeze(), self.label_mapping.index(str(row["Ground_Truth"]))

In [ ]:
# !ls /content/drive/MyDrive/floodnet_data/Images/Train_Image | wc -l

In [ ]:
qdict = json.load(open("/content/drive/MyDrive/floodnet_data/Questions/Training Question.json", "r"))

def get_train_val_splits(jsondict):
    total = len(jsondict.keys())
    train, val = train_test_split(list(jsondict.keys()), test_size=0.2)

    print(len(train),len(val))

    train_dict = dict()
    val_dict = dict()
    for i in train:
        train_dict[str(i)] = jsondict[str(i)]
    
    for j in val:
        val_dict[str(j)] = jsondict[str(j)]
    
    return train_dict, val_dict

def get_train_val_splits_typewise(jsondict):
    simple_count = dict()
    complex_count = dict()
    yes_no = dict()
    entire_image = dict()
    road_condition = dict()

    counter = 0
    for example in jsondict.values():
        if example["Question_Type"] == "Yes_No":
            yes_no[str(counter)] = example
            counter += 1
        elif example["Question_Type"] == "Simple_Counting":
            simple_count[str(counter)] = example
            counter += 1
        elif example["Question_Type"] == "Complex_Counting":
            complex_count[str(counter)] = example
            counter += 1
        elif "road" in example["Question"]:
            road_condition[str(counter)] = example
            counter += 1
        elif "overall" in example["Question"]:
            entire_image[str(counter)] = example
            counter += 1

    train_dict = dict()
    val_dict = dict()

    for typedict in [road_condition,simple_count,entire_image,complex_count,yes_no]:
        
        print(len(typedict.keys()))
        train, val = train_test_split(list(typedict.keys()), test_size=0.2)
        for idx, i in enumerate(train):
            
            if idx == 0:
                print(typedict[str(i)])
            train_dict[str(i)] = typedict[str(i)]
        
        for idx, j in enumerate(val):
            
            if idx == 0:
                print(typedict[str(j)])
            val_dict[str(j)] = typedict[str(j)]
        


    return train_dict, val_dict

def get_uniq_image_ids(jsondict):
    uniq_images = []
    for key, example in jsondict.items():
        if example["Image_ID"] not in uniq_images:
            uniq_images.append(example["Image_ID"])
    return uniq_images

def get_questions_for_imageid(jsondict, imageid):
    qs = dict()
    for key, example in jsondict.items():
        if example["Image_ID"] == imageid:
            qs[key] = example
    return qs

def get_train_val_splits_imagewise(jsondict):
    train_dict = dict()
    val_dict = dict()

    uniq_images = get_uniq_image_ids(jsondict)
    
    train, val = train_test_split(uniq_images, test_size=0.2)

    for imageid in train:
        train_dict.update(get_questions_for_imageid(jsondict, imageid)) 

    for imageid in val:
        val_dict.update(get_questions_for_imageid(jsondict, imageid))

    return train_dict, val_dict

def get_typewise_train_val_splits(train_dict, val_dict):
    train_road_condition = dict()
    train_yes_no = dict()
    train_image_condition = dict()

    counter = 0
    for example in train_dict.values():
        if example["Question_Type"] == "Yes_No":
            train_yes_no[str(counter)] = example
            counter += 1
        elif "road" in example["Question"]:
            train_road_condition[str(counter)] = example
            counter += 1
        elif "overall" in example["Question"]:
            train_image_condition[str(counter)] = example
            counter += 1

    val_road_condition = dict()
    val_yes_no = dict()
    val_image_condition = dict()

    counter = 0
    for example in val_dict.values():
        if example["Question_Type"] == "Yes_No":
            val_yes_no[str(counter)] = example
            counter += 1
        elif "road" in example["Question"]:
            val_road_condition[str(counter)] = example
            counter += 1
        elif "overall" in example["Question"]:
            val_image_condition[str(counter)] = example
            counter += 1

    return [train_image_condition, train_road_condition, train_yes_no], [val_image_condition, val_road_condition, val_yes_no]


train_dict, val_dict  = get_train_val_splits_imagewise(qdict)
train_tasks, val_tasks = get_typewise_train_val_splits(train_dict, val_dict)

In [ ]:
print(len(train_dict.keys()))
print(len(val_dict.keys()))

3621
890


In [ ]:
train_ims = []
val_ims = []

for i in train_dict.values():
    if i["Image_ID"] not in train_ims:
        train_ims.append(i["Image_ID"])


for i in val_dict.values():
    if i["Image_ID"] not in val_ims:
        val_ims.append(i["Image_ID"])

In [ ]:
len(train_ims)

1158

In [ ]:
len(val_ims)

290

In [ ]:
def accuracy(pred, true):
    acc = np.sum((true == pred.argmax(-1)).astype(np.float32))
    return float(100 * acc / len(true))


In [ ]:
!mkdir ckpts

### Models


In [ ]:
from datetime import datetime

class VQAModelConcat(nn.Module):
    def __init__(self):
        super().__init__()
        self.linstack = nn.Sequential(
            # nn.Linear(in_features=3072, out_features=1024), # 3072 = 1024 (text) + 2048 (image)
            nn.Linear(in_features=1536, out_features=1024), # 3072 = 1024 (text) + 1536 (image)
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1024, out_features=512),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=256),
            nn.Linear(in_features=256, out_features=128),
            nn.Linear(in_features=128, out_features=56),
        )
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    def forward(self, batch):
        x = torch.concat([batch[0].float(), batch[1].float()], dim=-1)
        x = self.linstack(x)
        return x

    def fit(self, train_ds, val_ds):
        now = datetime.now()
        timestr = now.strftime("%d_%m_%Hh%Mm%Ss")
        os.makedirs(f"./ckpts/{timestr}", exist_ok=True)
        self.to(self.device)
        train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
        val_dl = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4)

        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        loss_fn = nn.CrossEntropyLoss()

        for epoch in range(100):
            print(f"########## Epoch {epoch}")
            self.train()
            optimizer.zero_grad()
            
            epoch_loss = []
            epoch_acc = []
            
            val_loss = []
            val_acc = []

            for batch in tqdm.tqdm(train_dl):
                batch = [elem.to(self.device) for elem in batch]
        #         batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())
                
                epoch_loss.append(loss.detach().cpu().numpy())
                epoch_acc.append(acc)
            
            self.eval()
            with torch.no_grad():
                for batch in tqdm.tqdm(val_dl):
                    batch = [elem.to(self.device) for elem in batch]
        #             batch[0] = tr(batch[0])
                    outputs = self(batch)
                    loss = loss_fn(outputs, batch[2])
            
                    acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                    val_loss.append(loss.detach().cpu().numpy())
                    val_acc.append(acc)
            
            
            torch.save(self.state_dict(), f"./ckpts/{timestr}/vqa_{epoch}.pt")
            
            print(f"Train loss: {np.mean(epoch_loss)}", end="\t")
            print(f"Val loss: {np.mean(val_loss)}", end="\t")
            print(f"Train acc: {np.mean(epoch_acc)}", end="\t")
            print(f"Val acc: {np.mean(val_acc)}")
    
    def evaluate(self, val_ds):
        val_dl = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)
        loss_fn = nn.CrossEntropyLoss()
        self.eval()
        self.to(self.device)
        val_loss = []
        val_acc = []
        with torch.no_grad():
            for batch in tqdm.tqdm(val_dl):
                batch = [elem.to(self.device) for elem in batch]
    #             batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
        
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                val_loss.append(loss.detach().cpu().numpy())
                val_acc.append(acc)
        print(f"Val loss: {np.mean(val_loss)}", end="\t")
        print(f"Val acc: {np.mean(val_acc)}")

In [ ]:
from datetime import datetime

class VQAModelAdd(nn.Module):
    def __init__(self):
        super().__init__()
        self.image = nn.Sequential(
         nn.Linear(in_features=768, out_features=1024),
         nn.Dropout(p=0.2),
         nn.Linear(in_features=1024, out_features=512)   
        )
        self.txt = nn.Linear(in_features=768, out_features=512)
        self.linstack = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=128),
            nn.Linear(in_features=128, out_features=56),
        )
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    def forward(self, batch):
        img = self.image(batch[0].float())
        txt = self.txt(batch[1].float())
        vec = img+txt
        # vec = torch.dot(img, txt) # check if they are 1-d tensors
        x = self.linstack(vec)
        return x

    def fit(self, train_ds, val_ds):
        now = datetime.now()
        timestr = now.strftime("%d_%m_%Hh%Mm%Ss")
        os.makedirs(f"./ckpts/{timestr}", exist_ok=True)
        self.to(self.device)
        train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
        val_dl = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4)

        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        loss_fn = nn.CrossEntropyLoss()

        best_val_acc = -1.

        for epoch in range(100):
            print(f"########## Epoch {epoch}")
            self.train()
            optimizer.zero_grad()
            
            epoch_loss = []
            epoch_acc = []
            
            val_loss = []
            val_acc = []

            for batch in tqdm.tqdm(train_dl):
                batch = [elem.to(self.device) for elem in batch]
        #         batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())
                
                epoch_loss.append(loss.detach().cpu().numpy())
                epoch_acc.append(acc)
            
            self.eval()
            with torch.no_grad():
                for batch in tqdm.tqdm(val_dl):
                    batch = [elem.to(self.device) for elem in batch]
        #             batch[0] = tr(batch[0])
                    outputs = self(batch)
                    loss = loss_fn(outputs, batch[2])
            
                    acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                    val_loss.append(loss.detach().cpu().numpy())
                    val_acc.append(acc)
            
            
            torch.save(self.state_dict(), f"./ckpts/{timestr}/vqa_{epoch}.pt")
            
            if np.mean(val_acc) > best_val_acc:
                best_val_acc = np.mean(val_acc)
                best_epoch = epoch


            print(f"Train loss: {np.mean(epoch_loss)}", end="\t")
            print(f"Val loss: {np.mean(val_loss)}", end="\t")
            print(f"Train acc: {np.mean(epoch_acc)}", end="\t")
            print(f"Val acc: {np.mean(val_acc)}")
    
    def evaluate(self, val_ds):
        val_dl = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)
        loss_fn = nn.CrossEntropyLoss()
        self.eval()
        self.to(self.device)
        val_loss = []
        val_acc = []
        with torch.no_grad():
            for batch in tqdm.tqdm(val_dl):
                batch = [elem.to(self.device) for elem in batch]
    #             batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
        
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                val_loss.append(loss.detach().cpu().numpy())
                val_acc.append(acc)
        print(f"Val loss: {np.mean(val_loss)}", end="\t")
        print(f"Val acc: {np.mean(val_acc)}")

In [ ]:
from datetime import datetime

class VQAModelProduct(nn.Module):
    def __init__(self):
        super().__init__()
        self.image = nn.Sequential(
         nn.Linear(in_features=768, out_features=1024),
         nn.Dropout(p=0.2),
         nn.Linear(in_features=1024, out_features=512)   
        )
        self.txt = nn.Linear(in_features=768, out_features=512)
        self.linstack = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=128),
            nn.Linear(in_features=128, out_features=56),
        )
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    def forward(self, batch):
        img = self.image(batch[0].float())
        txt = self.txt(batch[1].float())
        vec = img*txt
        # vec = torch.dot(img, txt) # check if they are 1-d tensors
        x = self.linstack(vec)
        return x

    def fit(self, train_ds, val_ds):
        now = datetime.now()
        timestr = now.strftime("%d_%m_%Hh%Mm%Ss")
        os.makedirs(f"./ckpts/{timestr}", exist_ok=True)
        self.to(self.device)
        train_dl = DataLoader(train_ds, batch_size=256, shuffle=True, num_workers=4)
        val_dl = DataLoader(val_ds, batch_size=256, shuffle=False, num_workers=4)

        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        loss_fn = nn.CrossEntropyLoss()

        for epoch in range(100):
            print(f"########## Epoch {epoch}")
            self.train()
            optimizer.zero_grad()
            
            epoch_loss = []
            epoch_acc = []
            
            val_loss = []
            val_acc = []

            best_val_acc = -1.

            for batch in tqdm.tqdm(train_dl):
                batch = [elem.to(self.device) for elem in batch]
        #         batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())
                
                epoch_loss.append(loss.detach().cpu().numpy())
                epoch_acc.append(acc)
            
            self.eval()
            with torch.no_grad():
                for batch in tqdm.tqdm(val_dl):
                    batch = [elem.to(self.device) for elem in batch]
        #             batch[0] = tr(batch[0])
                    outputs = self(batch)
                    loss = loss_fn(outputs, batch[2])
            
                    acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                    val_loss.append(loss.detach().cpu().numpy())
                    val_acc.append(acc)
            
            
            torch.save(self.state_dict(), f"./ckpts/{timestr}/vqa_{epoch}.pt")

            if np.mean(val_acc) > best_val_acc:
                best_val_acc = np.mean(val_acc)
                best_epoch = epoch
            
            print(f"Train loss: {np.mean(epoch_loss)}", end="\t")
            print(f"Val loss: {np.mean(val_loss)}", end="\t")
            print(f"Train acc: {np.mean(epoch_acc)}", end="\t")
            print(f"Val acc: {np.mean(val_acc)}")
        print(best_val_acc, best_epoch)
    
    def evaluate(self, val_ds):
        val_dl = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)
        loss_fn = nn.CrossEntropyLoss()
        self.eval()
        self.to(self.device)
        val_loss = []
        val_acc = []
        with torch.no_grad():
            for batch in tqdm.tqdm(val_dl):
                batch = [elem.to(self.device) for elem in batch]
    #             batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
        
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                val_loss.append(loss.detach().cpu().numpy())
                val_acc.append(acc)
        print(f"Val loss: {np.mean(val_loss)}", end="\t")
        print(f"Val acc: {np.mean(val_acc)}")
        print()

### Runs

890

In [ ]:
# train_image_condition, train_road_condition, train_yes_no


for train_dict, val_dict in list(zip(train_tasks, val_tasks)):
    train_ds = VQADataset(train_dict)
    val_ds = VQADataset(val_dict)
    len(val_ds)
    # model = VQAModelConcat()
    model = VQAModelProduct()
    # model = VQAModelAdd()
    # model = VQAModelAttention()
    model.fit(train_ds, val_ds)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


########## Epoch 0


100%|██████████| 2/2 [00:00<00:00, 10.23it/s]


Train loss: 3.8242969512939453	Val loss: 3.515929698944092	Train acc: 67.88129664179105	Val acc: 86.00643382352942
########## Epoch 1


100%|██████████| 2/2 [00:00<00:00, 10.46it/s]


...

Train loss: 0.009692427702248096	Val loss: 0.07626555114984512	Train acc: 99.55439814814815	Val acc: 97.71428571428571
########## Epoch 98


100%|██████████| 1/1 [00:00<00:00,  5.00it/s]


Train loss: 0.01003097090870142	Val loss: 0.07437101006507874	Train acc: 99.55439814814815	Val acc: 97.71428571428571
########## Epoch 99


100%|██████████| 1/1 [00:00<00:00,  4.43it/s]

Train loss: 0.009012733586132526	Val loss: 0.07280134409666061	Train acc: 99.68460648148148	Val acc: 97.71428571428571
97.71428571428571 99


In [ ]:
# a = torch.tensor([1, 1, 2, 1])
# b = torch.tensor([11, 12, 3, 2])
# a*b

In [ ]:
simple_count = dict()
complex_count = dict()
yes_no = dict()
entire_image = dict()
road_condition = dict()

counter = 0
for example in val_dict.values():
    if example["Question_Type"] == "Yes_No":
        yes_no[str(counter)] = example
        counter += 1
    elif example["Question_Type"] == "Simple_Counting":
        simple_count[str(counter)] = example
        counter += 1
    elif example["Question_Type"] == "Complex_Counting":
        complex_count[str(counter)] = example
        counter += 1
    elif "road" in example["Question"]:
        road_condition[str(counter)] = example
        counter += 1
    elif "overall" in example["Question"]:
        entire_image[str(counter)] = example
        counter += 1

simple_count = VQADataset(simple_count)
complex_count = VQADataset(complex_count)
road_condition = VQADataset(road_condition)
yes_no = VQADataset(yes_no)
entire_image = VQADataset(entire_image)

train_ds = VQADataset(train_dict)
val_ds = VQADataset(val_dict)
len(val_ds)

890

In [ ]:
print(len(simple_count))
print(len(complex_count))
print(len(yes_no))
print(len(entire_image))
print(len(road_condition))

120
130
175
290
175


In [ ]:
!ls /content/ckpts/

20_09_16h43m15s  20_09_16h52m49s  20_09_17h28m06s  20_09_17h42m01s  best.pt
20_09_16h43m46s  20_09_16h57m08s  20_09_17h29m52s  20_09_17h46m10s
20_09_16h44m42s  20_09_17h03m56s  20_09_17h40m18s  20_09_17h50m08s


In [ ]:
# model = VQAModelConcat()
# model = VQAModelAdd()
model = VQAModelProduct()
model.load_state_dict(torch.load("/content/ckpts/20_09_17h50m08s/vqa_47.pt"))
model.eval()

VQAModelProduct(
  (image): Sequential(
    (0): Linear(in_features=768, out_features=1024, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=1024, out_features=512, bias=True)
  )
  (txt): Linear(in_features=768, out_features=512, bias=True)
  (linstack): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=56, bias=True)
  )
)

In [ ]:
model.evaluate(val_ds)


100%|██████████| 890/890 [00:03<00:00, 286.20it/s]

Val loss: 0.6203692555427551	Val acc: 81.79775280898876



In [ ]:
model.evaluate(simple_count)


100%|██████████| 120/120 [00:00<00:00, 172.58it/s]

Val loss: 1.714484453201294	Val acc: 46.666666666666664



In [ ]:
model.evaluate(complex_count)


100%|██████████| 130/130 [00:00<00:00, 188.91it/s]

Val loss: 2.0441653728485107	Val acc: 42.30769230769231



In [ ]:
model.evaluate(yes_no)


100%|██████████| 175/175 [00:00<00:00, 207.51it/s]

Val loss: 0.07158318907022476	Val acc: 97.71428571428571



In [ ]:
model.evaluate(entire_image)


100%|██████████| 290/290 [00:01<00:00, 242.30it/s]

Val loss: 0.13860592246055603	Val acc: 95.17241379310344



In [ ]:
model.evaluate(road_condition)


100%|██████████| 175/175 [00:00<00:00, 210.65it/s]

Val loss: 0.15957875549793243	Val acc: 97.14285714285714



In [ ]:
# !pip install transformers
# # from transformers import ViLT

In [ ]:
# from transformers import ViltModel

In [ ]:
# from transformers import ViltProcessor, ViltModel
# from PIL import Image
# import requests

# # prepare image and text
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
# text = "hello world"

# processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")
# model = ViltModel.from_pretrained("dandelin/vilt-b32-mlm")

# inputs = processor(image, text, return_tensors="pt")
# outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state

In [ ]:
# from prettytable import PrettyTable

# def count_parameters(model):
#     table = PrettyTable(["Modules", "Parameters"])
#     total_params = 0
#     for name, parameter in model.named_parameters():
#         if not parameter.requires_grad: continue
#         params = parameter.numel()
#         table.add_row([name, params])
#         total_params+=params
#     print(table)
#     print(f"Total Trainable Params: {total_params}")
#     return total_params
    
# count_parameters(model)

In [ ]:
# from datetime import datetime

# class AttentionBlock(nn.Module):
#   def __init__(self, embed_dim=512, num_heads=4):
#     super().__init__()
#     self.multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
#     self.lnorm = nn.LayerNorm(embed_dim)
#     self.feed_forward = nn.Linear(embed_dim, embed_dim)

#   def forward(self, k, v, q):
#     attn_output, attn_output_weights = self.multihead_attn(query=q, key=k, value=v)
#     attn_output = self.lnorm(attn_output + q)
#     attn_output = self.lnorm(self.feed_forward(attn_output) + attn_output)
#     return attn_output


# class VQAModelAttention(nn.Module):
#     def __init__(self, embed_dim=512, num_heads=4, num_blocks=1):
#         super().__init__()
#         self.attn = nn.ModuleList([
#                                *(AttentionBlock(embed_dim=512, num_heads=4) for _ in range(num_blocks))
#                                 ])

#         self.image = nn.Sequential(
#          nn.Linear(in_features=2048, out_features=1024),
#          nn.Dropout(p=0.2),
#          nn.Linear(in_features=1024, out_features=512)   
#         )
#         self.txt = nn.Linear(in_features=1024, out_features=512)
        
#         self.linstack = nn.Sequential(
#             nn.Linear(in_features=512, out_features=256),
#             nn.Dropout(p=0.2),
#             nn.Linear(in_features=256, out_features=128),
#             nn.Linear(in_features=128, out_features=56),
#         )
#         self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
#     def forward(self, batch):
#         img = self.image(batch[0])
#         txt = self.txt(batch[1])
        
#         attn_output = img
#         for layer in self.attn:
#           attn_output = layer(k=attn_output, v=attn_output, q=txt)
          
#         x = self.linstack(attn_output)
#         return x

#     def fit(self, train_ds, val_ds):
#         now = datetime.now()
#         timestr = now.strftime("%d_%m_%Hh%Mm%Ss")
#         os.makedirs(f"./ckpts/{timestr}", exist_ok=True)
#         self.to(self.device)
#         train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
#         val_dl = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4)

#         optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
#         loss_fn = nn.CrossEntropyLoss()

#         for epoch in range(100):
#             print(f"########## Epoch {epoch}")
#             self.train()
#             optimizer.zero_grad()
            
#             epoch_loss = []
#             epoch_acc = []
            
#             val_loss = []
#             val_acc = []

#             for batch in tqdm.tqdm(train_dl):
#                 batch = [elem.to(self.device) for elem in batch]
#         #         batch[0] = tr(batch[0])
#                 outputs = self(batch)
#                 loss = loss_fn(outputs, batch[2])
#                 loss.backward()
#                 optimizer.step()
#                 optimizer.zero_grad()
                
#                 acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())
                
#                 epoch_loss.append(loss.detach().cpu().numpy())
#                 epoch_acc.append(acc)
            
#             self.eval()
#             with torch.no_grad():
#                 for batch in tqdm.tqdm(val_dl):
#                     batch = [elem.to(self.device) for elem in batch]
#         #             batch[0] = tr(batch[0])
#                     outputs = self(batch)
#                     loss = loss_fn(outputs, batch[2])
            
#                     acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

#                     val_loss.append(loss.detach().cpu().numpy())
#                     val_acc.append(acc)
            
            
#             torch.save(self.state_dict(), f"./ckpts/{timestr}/vqa_{epoch}.pt")
            
#             print(f"Train loss: {np.mean(epoch_loss)}", end="\t")
#             print(f"Val loss: {np.mean(val_loss)}", end="\t")
#             print(f"Train acc: {np.mean(epoch_acc)}", end="\t")
#             print(f"Val acc: {np.mean(val_acc)}")
    
#     def evaluate(self, val_ds):
#         val_dl = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)
#         loss_fn = nn.CrossEntropyLoss()
#         self.eval()
#         self.to(self.device)
#         val_loss = []
#         val_acc = []
#         with torch.no_grad():
#             for batch in tqdm.tqdm(val_dl):
#                 batch = [elem.to(self.device) for elem in batch]
#     #             batch[0] = tr(batch[0])
#                 outputs = self(batch)
#                 loss = loss_fn(outputs, batch[2])
        
#                 acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

#                 val_loss.append(loss.detach().cpu().numpy())
#                 val_acc.append(acc)
#         print(f"Val loss: {np.mean(val_loss)}", end="\t")
#         print(f"Val acc: {np.mean(val_acc)}")